In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%matplotlib inline
import imp

from matplotlib import pyplot as plt

try:
    imp.find_module('jupyterplot')
    from jupyterplot import ProgressPlot
except ImportError:
    !pip install jupyterplot
    from jupyterplot import ProgressPlot

import torch
from torch import nn

import torchvision
from torchvision import datasets as D
from torchvision import transforms as T

from utils import invest_size
from utils import sample_random_data
from utils import show_images

# Convolutional Neural Networks for Classification

![](https://www.researchgate.net/publication/332284670/figure/fig1/AS:745591005007874@1554774156651/Example-of-a-CNN-for-image-classification.ppm)

# Image ? Feature map?
## Image
![](https://res.cloudinary.com/practicaldev/image/fetch/s--7uHGwEG8--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_880/https://i.ibb.co/HgnybWG/rgb.png)
![](https://res.cloudinary.com/practicaldev/image/fetch/s--BXoVOWNw--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_880/https://i.ibb.co/yyDtW47/own2d.png)
![](https://res.cloudinary.com/practicaldev/image/fetch/s--L7_r7KuE--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_880/https://i.ibb.co/hWdkRpd/last.png)

# Feature map

Neural network의 각 레이어가 다음 레이어로 전달하는 정보

![](https://miro.medium.com/max/700/1*LlRAQHT0ktl_33VUnDhoIg.png)

# Convolution layer

![](https://a.disquscdn.com/get?url=http%3A%2F%2Fi.imgur.com%2FOc1zZOM.png&key=LkQy1acZbmFHr9m1gWJUvA&w=800&h=387)

- 컨볼루션 레이어의 stride, padding을 변화시키며 출력 feature의 크기를 확인해보기

#### $N_{out}=\lfloor \frac{N_{in}+2\cdot padding-kernel size}{stride} \rfloor+1$

##### ex) $stride=1$, $kernelsize=3$, $padding=1$, 컨볼루션 레이어에
##### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$5\times 5$ 크기의 입력이 들어가면
##### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$5\times 5$ 크기의 출력이 나옴
![](https://theano-pymc.readthedocs.io/en/latest/_images/same_padding_no_strides.gif)

##### ex) $stride=2$, $kernelsize=3$, $padding=1$, 컨볼루션 레이어에
##### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$6\times 6$ 크기의 입력이 들어가면
##### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$3\times 3$ 크기의 출력이 나옴
![](https://theano-pymc.readthedocs.io/en/latest/_images/padding_strides_odd.gif)

https://theano-pymc.readthedocs.io/en/latest/tutorial/conv_arithmetic.html

In [ ]:
batch_size = 1
num_channels = 1
size_W = 24
size_H = 24

padding = 1
stride = 2
kernel_size = 3
out_channels = 1

inputs = torch.randn(batch_size, num_channels, size_W, size_H)

convolution_layer = nn.Conv2d(
    in_channels=num_channels,
    out_channels=out_channels,
    kernel_size=kernel_size,
    stride=stride,
    padding=padding
)
invest_size(inputs, convolution_layer)

# Max Pooling Layer
![](https://media.vlpt.us/images/tmddn0311/post/2abc7701-340d-494f-880c-01238a3439b4/image.png)

In [ ]:
kernel_size = 2
stride = 2
padding = 0

inputs = torch.tensor([[
    [12, 20, 30, 0],
    [8, 12, 2, 0],
    [34, 70, 37, 4],
    [112, 100, 25, 12]
]], dtype=torch.float)

maxpooling_layer = nn.MaxPool2d(
    kernel_size=kernel_size,
    stride=stride,
    padding=padding
)

invest_size(inputs, maxpooling_layer)
print(maxpooling_layer(inputs))

# Convolution Neural Networks 만들기

In [ ]:
class ConvolutionNeuralNetworks(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=16,
            kernel_size=3, stride=1, padding=1
        )
        self.relu1 = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.relu2 = nn.ReLU(inplace=True)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(32, 64, 3, 1, 1)
        self.relu3 = nn.ReLU(inplace=True)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(in_features=64 * 4 * 4, out_features=64)
        self.fc2 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        
        x = x.flatten(start_dim=1)
        
        x = self.fc1(x)
        x = self.fc2(x)
        return x
cnn = ConvolutionNeuralNetworks()
print(cnn)

# MNIST Dataset
![](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

### 데이터셋 생성

In [ ]:
data_transform = T.Compose([T.Resize((32, 32)), T.ToTensor()])
mnist_dataset = {
    'train': D.MNIST(
        root='data',
        train=True,
        transform=data_transform,
        download=True
    ),
    'test': D.MNIST(
        root='data',
        train=False,
        transform=data_transform
    )
}
print(f'훈련 집합의 데이터 개수: {len(mnist_dataset["train"])}')
print(f'테스트 집합의 데이터 개수: {len(mnist_dataset["test"])}')

### 데이터 시각화

In [ ]:
images, targets = sample_random_data(mnist_dataset['train'])
show_images(images, targets)

# 훈련시키기

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 5
batch_size = 64
learning_rate = 0.005
momentum = 0.9

In [ ]:
cnn = ConvolutionNeuralNetworks().to(device)
optimizer = torch.optim.SGD(cnn.parameters(), lr=learning_rate, momentum=momentum)
loss_function = torch.nn.CrossEntropyLoss()
data_loader = {
    'train': torch.utils.data.DataLoader(
        dataset=mnist_dataset['train'],
        batch_size=batch_size,
        shuffle=True,
    ),
    'test': torch.utils.data.DataLoader(
        dataset=mnist_dataset['test'],
        batch_size=batch_size,
        shuffle=False
    )
}

print(f'한 epoch 당 iteration 수: {len(mnist_dataset["train"])} / {batch_size} = {len(data_loader["train"])}')

loss_basket = []
accuracy_basket = []

pp = ProgressPlot(
    plot_names=['train', 'test'],
    line_names=['loss', 'accuracy'],
    x_lim=[0, num_epochs*len(data_loader['train'])],
    x_label='Iteration',
    y_lim=[[0, 2.5], [95, 100]]
)
accuracy = 0
for epoch in range(num_epochs):
    cnn.train()
    torch.set_grad_enabled(True)
    for iteration, (inputs, target) in enumerate(data_loader['train']):
        optimizer.zero_grad()
        inputs, target = inputs.to(device), target.to(device)
        
        output = cnn(inputs)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        
        loss_basket.append(loss.item())
        pp.update([[loss.item(), -1], [0, accuracy]])
    
    cnn.eval()
    corrects = 0
    torch.set_grad_enabled(False)
    for inputs, target in data_loader['test']:
        inputs, target = inputs.to(device), target.to(device)
        
        output = cnn(inputs)
        scores, predicted_classes = output.max(dim=1)
        
        corrects += (predicted_classes == target).sum().item()
        
    accuracy = corrects/len(mnist_dataset["test"])*100
    accuracy_basket.append(accuracy)
    print(f'Epoch: {epoch+1} accuracy {accuracy:.2f}')
pp.finalize()

In [ ]:
plt.plot(loss_basket)
plt.xlabel('iteration')
plt.ylabel('loss')
plt.show()

plt.plot(accuracy_basket)
plt.xlabel('epoch')
plt.ylabel('accuracy %')
plt.show()

# 훈련 시킨 모델 테스트
- 반복적으로 실행하여 틀린 샘플 찾아보기

In [ ]:
images, targets = sample_random_data(mnist_dataset['test'], num=25)

scores = cnn(images.to(device))
predicted = scores.argmax(1).detach().cpu()
correctness = (predicted == torch.as_tensor(targets))
titles = [f'target {t}\npredicted as {p}\n{"correct" if c else "WRONG"}'
          for t, p, c in zip(targets, predicted, correctness)]
show_images(images, titles)